In [59]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as ET

periode=6 #-------------------------------

tree = ET.parse(f'XMLs\\periode_{periode-1}.xml')
root = tree.getroot()

base_df=pd.read_csv('Data_pl\\base_table_01.csv',index_col='Unnamed: 0')
base_df['Lieferfrist']=base_df['Lieferfrist'].str.replace(',','.').astype('float')
base_df['Abweichung']=base_df['Abweichung'].str.replace(',','.').astype('float')

dfl=[]
for a in root.find('warehousestock').findall('article'):
    dfl.append(a.attrib)
bestands_df = pd.DataFrame(dfl) 

for col in bestands_df.columns:
    try:
        bestands_df[col]=bestands_df[col].astype(int)
    except:
        bestands_df[col]=bestands_df[col].str.replace(',','.')
        bestands_df[col]=bestands_df[col].astype(float)

dfl=[]
for a in root.find('futureinwardstockmovement').findall('order'):
    dfl.append(a.attrib)
futur_orders_df = pd.DataFrame(dfl)

for col in futur_orders_df.columns:
    futur_orders_df[col]=futur_orders_df[col].astype(int)

dfl=[]
for a in root.find('inwardstockmovement').findall('order'):
    dfl.append(a.attrib)
orders_in_df = pd.DataFrame(dfl)

for col in orders_in_df.columns:
    try:
        orders_in_df[col]=orders_in_df[col].astype(int)
    except:
        orders_in_df[col]=orders_in_df[col].str.replace(',','.')
        orders_in_df[col]=orders_in_df[col].astype(float)

In [60]:
periodes=list(range(periode,periode+4))

plan=[#----------------------------------------
['P1',25,150,200,200],
['P2',250,200,200,200],
['P3',100,100,150,150]]

plan=pd.DataFrame(plan,columns=['Pr']+periodes)
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)
plan

,6,7,8,9
Pr,,,,
P1,25,150,200,200
P2,250,200,200,200
P3,100,100,150,150


In [61]:
merge_1=pd.merge(base_df,bestands_df,left_on='NR. K',right_on='id').drop(columns=['id','startamount','pct','price','stockvalue'])

for col in periodes:
    merge_1[f'Bedarf_{col}']=merge_1[['P1','P2','P3']]@plan[col]

In [62]:
merge_2=pd.merge(futur_orders_df,base_df,left_on='article',right_on='NR. K')
merge_2['avg_order_time']=merge_2['Lieferfrist']+merge_2['Abweichung']
merge_2=merge_2.drop(columns=['id','NR. K','P1','P2','P3','Diskontmenge','Lieferfrist','Abweichung'])

merge_2['avg_order_time']=merge_2.apply(lambda x : x['avg_order_time'] if x['mode'] == 5 else x['avg_order_time']/2,axis=1)
merge_2['expected_arival_periode']=merge_2['orderperiod']+merge_2['avg_order_time']
merge_2['expected_arival_periode']=np.floor(merge_2['expected_arival_periode'])

In [63]:
# current_col=5
def order_sum_at_periode(x):
    global current_col
    return sum(merge_2[(merge_2['article']==x['NR. K'])&(merge_2['expected_arival_periode']==current_col)]['amount'])

In [64]:
amount_df=merge_1.copy()
current_col=periodes[0]

amount_df[(f'paa_{periodes[0]}','start')]=amount_df['amount']
amount_df[(f'paa_{periodes[0]}','bed')]=amount_df[f'Bedarf_{periodes[0]}']
amount_df[(f'paa_{periodes[0]}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1)

current_col=periodes[0]-1
amount_df[(f'paa_{periodes[0]}','o_in')]+=amount_df.apply(order_sum_at_periode,axis=1)

amount_df[(f'paa_{periodes[0]}','end')]=(amount_df['amount']-amount_df[f'Bedarf_{periodes[0]}']).apply(lambda x : 0 if x<=0 else x)
amount_df[(f'paa_{periodes[0]}','end')]+=amount_df[(f'paa_{periodes[0]}','o_in')]


for col in periodes[1:]:
    current_col=col
    amount_df[(f'paa_{col}','start')]=amount_df[(f'paa_{col-1}','end')]
    amount_df[(f'paa_{col}','bed')]=amount_df[f'Bedarf_{col}']
    amount_df[(f'paa_{col}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1) 
    amount_df[(f'paa_{col}','end')]=(amount_df[(f'paa_{col-1}','end')]-amount_df[f'Bedarf_{col}']).apply(lambda x : 0 if x<=0 else x)
    amount_df[(f'paa_{col}','end')]+=amount_df[(f'paa_{col}','o_in')]


In [65]:
l=[]
for i,x in enumerate(amount_df.columns):
    if type(x)!=tuple:
        l.append((x,''))
    else:
        l.append(x)

amount_df.columns=l
amount_df.columns=pd.MultiIndex.from_tuples(amount_df.columns)

In [66]:
out=amount_df[[f'paa_{x}' for x in periodes]]
out.set_index(amount_df['NR. K']).to_csv(f'Data_pl\\amount_periode_{periode}.csv')

In [67]:
def needed_order_period(x):
    for i,y in enumerate(x):
        if y ==0:
            periode=iter_out.columns[i][0].split('_')[1]
            return int(periode)
    return max(periodes)+1
    

In [68]:
l=[]
for x in out.columns:
    if (x[1]=='o_in')|(x[1]=='start'):
        l.append(x)
iter_out=out.drop(columns=l)

# needed_order_at_start_of_period
amount_df['need_order']=iter_out.apply(needed_order_period,axis=1)+1
# needed_to_order_at_start_of_period
amount_df['need_to_order']=np.floor(amount_df['need_order']-amount_df['Lieferfrist']-amount_df['Abweichung'])
amount_df['need_to_order_iff']=np.floor(amount_df['need_order']-(amount_df['Lieferfrist']+amount_df['Abweichung'])/2)

In [69]:
def min_order_amount(x):
    need_order=int( x['need_to_order'][0] )
    
    if need_order==periode:
        return 'n', x[f'Bedarf_{periode}'][0]
    
    elif need_order<periode:
        return 'f' , x[f'Bedarf_{periode}'][0]
    
    else:
        return None , 0 

In [70]:
amount_df['order_type'],amount_df['min_order_amount']=zip( *amount_df.apply(min_order_amount,axis=1) )

amount_df.to_csv(f'Order_history\\order_amount_{periode}.csv')

In [71]:
amount_df.drop(columns=[
            #(                'NR. K',      ''),
            (                'P1',      ''),
            (                'P2',      ''),
            (                'P3',      ''),
            #(      'Diskontmenge',      ''),
            (            'amount',      ''),
            (          f'Bedarf_{periode}',      ''),
            (          f'Bedarf_{periode+1}',      ''),
            (          f'Bedarf_{periode+2}',      ''),
            (          f'Bedarf_{periode+3}',      ''),
            ])

NR. K Lieferfrist Abweichung Diskontmenge  paa_6                       \
                                              start    bed   o_in    end   
0     21         1.8        0.4          300    275     25      0    250   
1     22         1.7        0.4          300    400    250      0    150   
2     23         1.2        0.2          300    340    100      0    240   
3     24         3.2        0.3         6100      5   2625   9425   9425   
4     25         0.9        0.2         3600   2000   1500      0    500   
5     27         0.9        0.2         1800   1040    750      0    290   
6     28         1.7        0.4         4500   4890   1950      0   2940   
7     32         2.1        0.5         2700      5   1125   4125   4125   
8     33         1.9        0.5          900    380    200      0    180   
9     34         1.6        0.3        22000  18080   7200      0  10880   
10    35         2.2        0.4         3600      0   1500   3600   3600   
11    36         1.2        0.1          900     35    375    475    475   
12    37         1.5        0.3          900    440    375      0     65   
13    38         1.7        0.4          300    650    375      0    275   
14    39         1.5        0.3         1800    830    750    950   1030   
15    40         1.7        0.2          900    690    375      0    315   
16    41         0.9        0.2          900    690    375      0    315   
17    42         1.2        0.3         1800   1380    750      0    630   
18    43         2.0        0.5         2700     95    375   2700   2700   
19    44         1.0        0.2          900    555   1125   1425   1425   
20    45         1.7        0.3          900    470    375      0     95   
21    46         0.9        0.3          900    470    375      0     95   
22    47         1.1        0.1          900    480    375      0    105   
23    48         1.0        0.2         1800    880    750      0    130   
24    52         1.6        0.4          600    570     50      0    520   
25    53         1.6        0.2        22000    120   1800  16200  16200   
26    57         1.7        0.3          600    100    500      0      0   
27    58         1.6        0.5        22000  26400  18200      0   8200   
28    59         0.7        0.2         1800    940    750      0    190   

    paa_7         ... paa_8 paa_9                   need_order need_to_order  \
    start    bed  ...   end start    bed o_in   end                            
0     250    150  ...   125   125    200    0     0         10           7.0   
1     150    200  ...     0     0    200    0     0          8           5.0   
2     240    100  ...     0     0    150    0     0          9           7.0   
3    9425   3150  ...  2425  2425   3850    0     0         10           6.0   
4     500   1800  ...     0     0   2200    0     0          8           6.0   
5     290    900  ...     0     0   1100    0     0          8           6.0   
6    2940   2200  ...     0     0   2700    0     0          9           6.0   
7    4125   1350  ...  1125  1125   1650    0     0         10           7.0   
8     180    200  ...     0     0    300    0     0          8           5.0   
9   10880   7200  ...     0     0  10800    0     0          9           7.0   
10   3600   1800  ...     0     0   2200    0     0          9           6.0   
11    475    450  ...     0     0    550    0     0          9           7.0   
12     65    450  ...     0     0    550    0     0          8           6.0   
13    275    450  ...     0     0    550    0     0          8           5.0   
14   1030    900  ...     0     0   1100    0     0          9           7.0   
15    315    450  ...     0     0    550    0     0          8           6.0   
16    315    450  ...     0     0    550    0     0          8           6.0   
17    630    900  ...     0     0   1100    0     0          8           6.0   
18   2700    450  ...  1700  1700    550    0 

In [72]:
amount_df[[('NR. K',''),('Diskontmenge',''),('order_type',''),('min_order_amount',''),]]

,NR. K,Diskontmenge,order_type,min_order_amount
,,,,
0,21,300,None,0.0
1,22,300,f,250.0
2,23,300,None,0.0
3,24,6100,n,2625.0
4,25,3600,n,1500.0
5,27,1800,n,750.0
6,28,4500,n,1950.0
7,32,2700,None,0.0
8,33,900,f,200.0
